<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Predicting-Results" data-toc-modified-id="Predicting-Results-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Predicting Results</a></span></li><li><span><a href="#Read-in-Results-and-Features" data-toc-modified-id="Read-in-Results-and-Features-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Read in Results and Features</a></span><ul class="toc-item"><li><span><a href="#Read-in-results-and-statistics-dataset" data-toc-modified-id="Read-in-results-and-statistics-dataset-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Read in results and statistics dataset</a></span></li><li><span><a href="#Read-in-Bookie-Odds-dataset" data-toc-modified-id="Read-in-Bookie-Odds-dataset-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Read in Bookie Odds dataset</a></span></li><li><span><a href="#Create-single-line-df" data-toc-modified-id="Create-single-line-df-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Create single line df</a></span></li><li><span><a href="#Feature-labelling" data-toc-modified-id="Feature-labelling-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Feature labelling</a></span><ul class="toc-item"><li><span><a href="#Split-out-seasons" data-toc-modified-id="Split-out-seasons-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Split out seasons</a></span></li><li><span><a href="#Create-target-goals-dataframe" data-toc-modified-id="Create-target-goals-dataframe-2.4.2"><span class="toc-item-num">2.4.2&nbsp;&nbsp;</span>Create target goals dataframe</a></span></li></ul></li></ul></li><li><span><a href="#Prep-Dataset" data-toc-modified-id="Prep-Dataset-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Prep Dataset</a></span></li><li><span><a href="#Fit-a-model" data-toc-modified-id="Fit-a-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Fit a model</a></span></li><li><span><a href="#Predicting-all-games" data-toc-modified-id="Predicting-all-games-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Predicting all games</a></span></li></ul></div>

# Predicting Results

ideas and code taken from here
https://dashee87.github.io/football/python/predicting-football-results-with-statistical-modelling/
and 
https://dashee87.github.io/football/python/predicting-football-results-with-statistical-modelling-dixon-coles-and-time-weighting/

In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import log_loss


from sklearn.preprocessing import LabelEncoder,StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn import ensemble, metrics, model_selection as ms, preprocessing, tree
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn import linear_model as lm
from sklearn import svm
from sklearn import neighbors
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB

import statsmodels.api as sm
import statsmodels.formula.api as smf
pd.set_option('display.max_columns', 50)

import datetime


from pathlib import Path
import os
from os import listdir
from os.path import isfile, join

# Read in Results and Features

## Read in results and statistics dataset

In [139]:
results = pd.read_csv(r"C:\\Users\Byron\Documents\Football Predictions\PremierLeaguePredictions\data\ff_us_1419.csv").drop("Unnamed: 0",axis=1)
results.tail()

season     gameweek            matchid  \
4351    2019  Gameweek 28  /matches/1059980/   
4352    2019  Gameweek 28  /matches/1059975/   
4353    2019  Gameweek 28  /matches/1059975/   
4354    2019  Gameweek 28  /matches/1059979/   
4355    2019  Gameweek 28  /matches/1059979/   

                                           fixture            matchdate  \
4351                          Watford vs Liverpool  2020-02-29 17:30:00   
4352                  Everton vs Manchester United  2020-03-01 14:00:00   
4353                  Everton vs Manchester United  2020-03-01 14:00:00   
4354  Tottenham Hotspur vs Wolverhampton Wanderers  2020-03-01 14:00:00   
4355  Tottenham Hotspur vs Wolverhampton Wanderers  2020-03-01 14:00:00   

                       f_Team  f_HmGame  f_Possession  \
4351                Liverpool         0          71.0   
4352                  Everton         1          43.1   
4353        Manchester United         0          56.9   
4354        Tottenham Hotspur         1          65.9   
4355  Wolverhampton Wanderers         0          34.1   

      f_Touches - Final Third  f_Touches - Penalty Area  \
4351                      276                        31   
4352                      182                        25   
4353                      189                        17   
4354                      189                        23   
4355                      128                        25   

      f_Passes - Final Third - Successful  f_Chances Created  f_Goals  \
4351                                  148                  5        0   
4352                                   55                  8        1   
4353                                   97                 11        1   
4354                                   82                  9        2   
4355                                   42                 10        3   

      f_Goals From Penalties  f_Goal Attempts  f_Shots - Inside Box  \
4351                       0                7                     3   
4352                       0               16                    12   
4353                       0               14                     5   
4354                       0               13                    10   
4355                       0               14                    11   

      f_Big Chances Total  f_Shots On Target  f_Goal Attempts In Box Conceded  \
4351                    0                  1                               10   
4352                    4                  5                                5   
4353                    1                  5                               12   
4354                    3                  5                               11   
4355                    4                  4                               10   

      f_Shots Conceded  f_Shots On Target Conceded  f_Big Chances Conceded  \
4351                14                           5                       6   
4352                14                           5                       1   
4353                16                           5                       4   
4354                14                           4                       4   
4355                13                           5                       3   

      f_xG Open Play  f_xG Expected Goals  f_xA Expected Assists  \
4351            0.20                 0.29                   0.58   
4352            1.32                 2.09                   0.92   
4353            1.07                 1.13                   0.54   
4354            1.61                 1.70                   0.42   
4355            2.18                 2.56                   0.48   

      f_Goals Conceded  f_xG Conceded  f_Touches - Final Third Conceded  \
4351                 3           2.34                               123   
4352                 1           1.13                               189   
4353                 1           2.09                               182   
4354                 3           2.56

## Read in Bookie Odds dataset

In [48]:
odds = pd.read_csv(r"C:\\Users\Byron\Documents\Football Predictions\PremierLeaguePredictions\data\bookie_odds.csv").drop("Unnamed: 0",axis=1)
odds.tail()

league  season        Date                             fixture  \
2183    EPL    2018  09/02/2019        Huddersfield Town vs Arsenal   
2184    EPL    2018  07/10/2018                   Fulham vs Arsenal   
2185    EPL    2018  24/04/2019  Wolverhampton Wanderers vs Arsenal   
2186    EPL    2018  02/09/2018             Cardiff City vs Arsenal   
2187    EPL    2019  21/10/2019         Sheffield United vs Arsenal   

              HomeTeam AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
2183      Huddersfield  Arsenal   1.0   2.0   A   0.0   2.0   A   
2184            Fulham  Arsenal   1.0   5.0   A   1.0   1.0   D   
2185            Wolves  Arsenal   3.0   1.0   H   3.0   0.0   H   
2186           Cardiff  Arsenal   2.0   3.0   A   1.0   1.0   D   
2187  Sheffield United  Arsenal   1.0   0.0   H   1.0   0.0   H   

      BookieHomeOdds  BookieDrawOdds  BookieAwayOdds  bookie  BookieHomeProb  \
2183            5.75            4.40            1.60  Bet365        0.173913   
2184            5.25            4.33            1.66  Bet365        0.190476   
2185            2.90            3.50            2.55  Bet365        0.344828   
2186            6.50            4.33            1.57  Bet365        0.153846   
2187            3.90            3.75            1.90  Bet365        0.256410   

      BookieDrawProb  BookieAwayProb  BookieMargin BookiePredResult  \
2183        0.227273        0.625000      2.618577                A   
2184        0.230947        0.602410      2.383271                A   
2185        0.285714        0.392157      2.269873                A   
2186        0.230947        0.636943      2.173571                A   
2187        0.266667        0.526316      4.939271                A   

      BookiePredResultProb  BookieFinalResultProb  BookiePredCorrect  
2183              0.625000               0.625000                  1  
2184              0.602410               0.602410                  1  
2185              0.392157               0.344828                  0  
2186              0.636943               0.636943                  1  
2187              0.526316               0.256410                  0

Clean the dataset and create a bookie dataset to join on to our modelling dataset later for comparison.

In [49]:
odds_clean = odds[["season","fixture","FTR","bookie","BookieHomeOdds","BookieDrawOdds"
                  ,"BookieAwayOdds","BookieHomeProb","BookieDrawProb","BookieAwayProb"]].copy()
odds_clean.tail()

season                             fixture FTR  bookie  BookieHomeOdds  \
2183    2018        Huddersfield Town vs Arsenal   A  Bet365            5.75   
2184    2018                   Fulham vs Arsenal   A  Bet365            5.25   
2185    2018  Wolverhampton Wanderers vs Arsenal   H  Bet365            2.90   
2186    2018             Cardiff City vs Arsenal   A  Bet365            6.50   
2187    2019         Sheffield United vs Arsenal   H  Bet365            3.90   

      BookieDrawOdds  BookieAwayOdds  BookieHomeProb  BookieDrawProb  \
2183            4.40            1.60        0.173913        0.227273   
2184            4.33            1.66        0.190476        0.230947   
2185            3.50            2.55        0.344828        0.285714   
2186            4.33            1.57        0.153846        0.230947   
2187            3.75            1.90        0.256410        0.266667   

      BookieAwayProb  
2183        0.625000  
2184        0.602410  
2185        0.392157  
2186        0.636943  
2187        0.526316

## Create single line df
We will use our results dataset to create a single line results dataset that contains the Result. This will be our target variable.

In [50]:
def home_teams(df,keep):
    outdf = df.copy()
    outdf = outdf.query("f_HmGame==1")
    outdf = outdf[keep]
    outdf=outdf.rename(columns={'f_Team': 'HomeTeam'})
    return outdf

def away_teams(df,keep):
    outdf = df.copy()
    outdf = outdf.query("f_HmGame==0")
    outdf = outdf[keep]
    outdf=outdf.rename(columns={'f_Team': 'AwayTeam'})
    return outdf

def fixture_scores(df,score_keep_vars):
    outdf = df.copy()
    outdf = outdf.query("f_HmGame==1")
    outdf = outdf[score_keep_vars]
    outdf=outdf.rename(columns={'f_Goals': 'HomeGoals',
                                'f_Goals Conceded': 'AwayGoals'})
    return outdf

def join_fixtures(home,away,scores,merge_vars):
    outdf= pd.merge(home,away,on=merge_vars)
    outdf = pd.merge(outdf,scores,on=merge_vars)
    return outdf
def final_outcome(fixtures,homegoals,awaygoals):
    outdf = fixtures.copy()
    outdf["Result"] = np.where(outdf[homegoals]>outdf[awaygoals],"H",
                              np.where(outdf[homegoals]<outdf[awaygoals],"A","D"))
    return outdf

In [51]:
keep_vars = ["season","matchid","gw_no","fixture","matchdate","f_Team"]
score_keep_vars = ["season","matchid","gw_no","fixture","matchdate","f_Goals","f_Goals Conceded"]
merge_vars = ["season","matchid","gw_no","fixture","matchdate"]
team_vars = ["HomeTeam","AwayTeam"]

In [52]:
hometeams = home_teams(results,keep_vars)
awayteams = away_teams(results,keep_vars)
scores = fixture_scores(results,score_keep_vars)
single_line = join_fixtures(hometeams,awayteams,scores,merge_vars)
single_line = final_outcome(single_line,"HomeGoals","AwayGoals")
single_line.tail()

season            matchid  gw_no  \
2173    2019  /matches/1059977/     28   
2174    2019  /matches/1059981/     28   
2175    2019  /matches/1059980/     28   
2176    2019  /matches/1059975/     28   
2177    2019  /matches/1059979/     28   

                                           fixture            matchdate  \
2173                   Newcastle United vs Burnley  2020-02-29 15:00:00   
2174                West Ham United vs Southampton  2020-02-29 15:00:00   
2175                          Watford vs Liverpool  2020-02-29 17:30:00   
2176                  Everton vs Manchester United  2020-03-01 14:00:00   
2177  Tottenham Hotspur vs Wolverhampton Wanderers  2020-03-01 14:00:00   

               HomeTeam                 AwayTeam  HomeGoals  AwayGoals Result  
2173   Newcastle United                  Burnley          0          0      D  
2174    West Ham United              Southampton          3          1      H  
2175            Watford                Liverpool          3          0      H  
2176            Everton        Manchester United          1          1      D  
2177  Tottenham Hotspur  Wolverhampton Wanderers          2          3      A

**List Features**

## Feature labelling

In [53]:
features = [i for i in results.columns if i.startswith("f_") and i !="f_Team"]
def_features = [i for i in features if "Conceded" in i]
att_features = [i for i in features if i not in def_features]
calculated_features = ["f_netxg","f_xGRatio","f_ShotsRatio","f_ShOnTargetRatio"]
print(att_features)
print()
print(def_features)
print()
print(calculated_features)

['f_HmGame', 'f_Possession', 'f_Touches - Final Third', 'f_Touches - Penalty Area', 'f_Passes - Final Third - Successful', 'f_Chances Created', 'f_Goals', 'f_Goals From Penalties', 'f_Goal Attempts', 'f_Shots - Inside Box', 'f_Big Chances Total', 'f_Shots On Target', 'f_xG Open Play', 'f_xG Expected Goals', 'f_xA Expected Assists', 'f_US xG', 'f_Points', 'f_Touches - Final Third log']

['f_Goal Attempts In Box Conceded', 'f_Shots Conceded', 'f_Shots On Target Conceded', 'f_Big Chances Conceded', 'f_Goals Conceded', 'f_xG Conceded', 'f_Touches - Final Third Conceded', 'f_Touches - Penalty Area Conceded', 'f_Passes - Final Third - Successful Conceded', 'f_xG Open Play Conceded', 'f_US xG Conceded', 'f_Chances Conceded', 'f_xA Expected Assists Conceded']

['f_netxg', 'f_xGRatio', 'f_ShotsRatio', 'f_ShOnTargetRatio']


In [54]:
df_null = results[features].isnull().sum()/results[features].count()

In [55]:
##handling nulls
df_null = results[features].isnull().sum()/results[features].count()
df_null = df_null[df_null>0]
null_vars = df_null.index.tolist()
print(null_vars)

results = results.drop(null_vars,axis=1)

['f_xA Expected Assists Conceded']


In [56]:
features = [i for i in results.columns if i.startswith("f_") and i !="f_Team"]
def_features = [i for i in features if "Conceded" in i]
att_features = [i for i in features if i not in def_features]
print(att_features)
print()
print(def_features)


['f_HmGame', 'f_Possession', 'f_Touches - Final Third', 'f_Touches - Penalty Area', 'f_Passes - Final Third - Successful', 'f_Chances Created', 'f_Goals', 'f_Goals From Penalties', 'f_Goal Attempts', 'f_Shots - Inside Box', 'f_Big Chances Total', 'f_Shots On Target', 'f_xG Open Play', 'f_xG Expected Goals', 'f_xA Expected Assists', 'f_US xG', 'f_Points', 'f_Touches - Final Third log']

['f_Goal Attempts In Box Conceded', 'f_Shots Conceded', 'f_Shots On Target Conceded', 'f_Big Chances Conceded', 'f_Goals Conceded', 'f_xG Conceded', 'f_Touches - Final Third Conceded', 'f_Touches - Penalty Area Conceded', 'f_Passes - Final Third - Successful Conceded', 'f_xG Open Play Conceded', 'f_US xG Conceded', 'f_Chances Conceded']


### Split out seasons

In [142]:
training = results[results["season"].isin([2014,2015,2016,2017])]
test = results[results["season"]==2018]
holdout = results[results["season"]==2019]

### Create target goals dataframe
This contains target variables for a per team modelling dataset predicting goals scored or conceded

In [58]:
target_goals = results[['f_Team', 'matchid','f_HmGame','f_Goals', 'f_Goals Conceded',"f_US xG", "f_US xG Conceded"]].copy() 
target_goals.columns = ['f_Team', 'matchid','f_HmGameFut','t_Goals', 't_Goals Conceded',"t_US xG", "t_US xG Conceded"]
target_goals.loc[0:,"t_Goals_Scored_In"] = np.where(target_goals["t_Goals"]>0,1,0)
target_goals.loc[0:,"t_Goals_Conceded_In"] = np.where(target_goals["t_Goals Conceded"]>0,1,0)
target_goals.head()

f_Team           matchid  f_HmGameFut  t_Goals  \
0       Swansea City  /matches/755307/            0        2   
1  Manchester United  /matches/755307/            1        1   
2            Everton  /matches/755305/            0        2   
3     Leicester City  /matches/755305/            1        2   
4          Hull City  /matches/755309/            0        1   

   t_Goals Conceded   t_US xG  t_US xG Conceded  t_Goals_Scored_In  \
0                 1  0.278076          1.166350                  1   
1                 2  1.166350          0.278076                  1   
2                 2  0.613273          1.278300                  1   
3                 2  1.278300          0.613273                  1   
4                 0  1.117570          1.900670                  1   

   t_Goals_Conceded_In  
0                    1  
1                    1  
2                    1  
3                    1  
4                    0

In [59]:
target_goals[target_goals["f_Team"]== "Everton"].nunique()

f_Team                   1
matchid                218
f_HmGameFut              2
t_Goals                  7
t_Goals Conceded         7
t_US xG                218
t_US xG Conceded       218
t_Goals_Scored_In        2
t_Goals_Conceded_In      2
dtype: int64

# Prep Dataset

In [60]:
test_2016 = training[training["season"]==2016][["season","gw_no","fixture","f_Team","f_HmGame","f_US xG"]].copy()
test_2016.columns = test_2016.columns.str.replace(' ', '_')
def id_opponent(fixture,home):
    if home==1:
        opponent = fixture.split(" vs ")[1]
    else:
        opponent = fixture.split(" vs ")[0]
    return opponent
        
test_2016["opponent"] = test_2016.apply(lambda row: id_opponent(row["fixture"],row["f_HmGame"]),axis=1)
test_2016.head()

season  gw_no                                 fixture  \
1520    2016      1             Hull City vs Leicester City   
1521    2016      1             Hull City vs Leicester City   
1522    2016      1                 Burnley vs Swansea City   
1523    2016      1                 Burnley vs Swansea City   
1524    2016      1  Crystal Palace vs West Bromwich Albion   

                    f_Team  f_HmGame   f_US_xG        opponent  
1520        Leicester City         0  2.456310       Hull City  
1521             Hull City         1  0.740018  Leicester City  
1522          Swansea City         0  1.720890         Burnley  
1523               Burnley         1  1.049030    Swansea City  
1524  West Bromwich Albion         0  0.684593  Crystal Palace

# Fit a model

In [110]:
poisson_model = smf.glm(formula="f_US_xG ~ f_HmGame + f_Team + opponent", data=test_2016, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                f_US_xG   No. Observations:                  760
Model:                            GLM   Df Residuals:                      720
Model Family:                 Poisson   Df Model:                           39
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -902.81
Date:                Thu, 18 Jun 2020   Deviance:                       237.44
Time:                        00:48:06   Pearson chi2:                     235.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            0.3520      0.199      1.773      0.076      -0.037       0.741
f_Team[T.Bournemouth]               -0.2234      0.189     -1.180      0.238      -0.595       0.148
f_Team[T.Burnley]                   -0.5730      0.210     -2.734      0.006      -0.984      -0.162
f_Team[T.Chelsea]                   -0.0484      0.179     -0.271      0.787      -0.399       0.302
f_Team[T.Crystal Palace]            -0.3346      0.195     -1.719      0.086      -0.716       0.047
f_Team[T.Everton]                   -0.2498      0.190     -1.317      0.188      -0.622       0.122
f_Team[T.Hull City]                 -0.6059      0.213     -2.843      0.004      -1.024      -0.188
f_Team[T.Leicester City]            -0.3569      0.196     -1.822      0.069      -0.741       0.027
f_Team[T.Liverpool]                  0.0368      0.176      0.209      0.834      -0.307       0.381
f_Team[T.Manchester City]            0.2095      0.168      1.245      0.213      -0.120       0.540
f_Team[T.Manchester United]         -0.1162      0.182     -0.638      0.524      -0.473       0.241
f_Team[T.Middlesbrough]             -0.7137      0.219     -3.251      0.001      -1.144      -0.283
f_Team[T.Southampton]               -0.2456      0.189     -1.298      0.194      -0.617       0.125
f_Team[T.Stoke City]                -0.2829      0.192     -1.476      0.140      -0.659       0.093
f_Team[T.Sunderland]                -0.6012      0.212     -2.831      0.005      -1.017      -0.185
f_Team[T.Swansea City]              -0.4580      0.203     -2.257      0.024      -0.856      -0.060
f_Team[T.Tottenham Hotspur]          0.0830      0.173      0.478      0.632      -0.257       0.423
f_Team[T.Watford]                   -0.5574      0.208     -2.675      0.007      -0.966      -0.149
f_Team[T.West Bromwich Albion]      -0.5096      0.205     -2.487      0.013      -0.911      -0.108
f_Team[T.West Ham United]           -0.2632      0.191     -1.376      0.169      -0.638       0.112
opponent[T.Bournemouth]              0.2776      0.193      1.438      0.150      -0.101       0.656
opponent[T.Burnley]                  0.1308      0.199      0.659      0.510      -0.258       0.520
opponent[T.Chelsea]                 -0.5009      0.237     -2.110      0.035      -0.966      -0.036
opponent[T.Crystal Palace]           0.0415      0.203      0.204      0.838      -0.357       0.440
opponent[T.Everton]                 -0.0429      0.208     -0.207      0.836      -0.450       0.365
opponent[T.Hull City]                0.3909      0.188      2.081      0.037       0.023       0.759
opponent[T.Leicester City]           0.0656      0.202      0.324      0.746      -0.331       0.462
opponent[T.Liverpool]               -0.2207      0.21

In [108]:
poisson_model.summary2().tables[1].reset_index()#loc[["f_Team[T.Everton]"]]

index     Coef.  Std.Err.         z     P>|z|  \
0                          Intercept  0.403669  1.030274  0.391808  0.695200   
1              f_Team[T.Aston Villa] -2.125010  1.256263 -1.691532  0.090735   
2                  f_Team[T.Burnley] -1.179032  1.159500 -1.016845  0.309227   
3                  f_Team[T.Chelsea]  0.149549  0.685732  0.218087  0.827362   
4           f_Team[T.Crystal Palace] -1.651379  1.252510 -1.318455  0.187351   
5                  f_Team[T.Everton] -0.373979  0.897920 -0.416495  0.677048   
6                f_Team[T.Hull City] -1.728792  1.296696 -1.333228  0.182457   
7           f_Team[T.Leicester City] -0.646158  0.928380 -0.696006  0.486425   
8                f_Team[T.Liverpool] -0.006805  1.031893 -0.006594  0.994739   
9          f_Team[T.Manchester City] -1.483768  1.262826 -1.174958  0.240011   
10       f_Team[T.Manchester United] -1.093672  1.080553 -1.012141  0.311471   
11        f_Team[T.Newcastle United] -0.160415  0.916883 -0.174957  0.861113   
12     f_Team[T.Queens Park Rangers] -0.993538  1.034713 -0.960207  0.336951   
13             f_Team[T.Southampton] -1.141709  1.228510 -0.929344  0.352711   
14              f_Team[T.Stoke City] -1.056819  1.171025 -0.902473  0.366806   
15              f_Team[T.Sunderland] -1.483293  1.262140 -1.175221  0.239906   
16            f_Team[T.Swansea City] -0.794395  1.099821 -0.722294  0.470114   
17       f_Team[T.Tottenham Hotspur] -1.406249  1.263823 -1.112695  0.265839   
18    f_Team[T.West Bromwich Albion] -0.859718  1.108492 -0.775574  0.438000   
19         f_Team[T.West Ham United] -0.461999  0.953736 -0.484410  0.628095   
20           opponent[T.Aston Villa] -0.406251  1.431406 -0.283813  0.776554   
21               opponent[T.Burnley]  0.160215  1.101828  0.145409  0.884388   
22               opponent[T.Chelsea] -0.028547  0.955677 -0.029871  0.976170   
23        opponent[T.Crystal Palace]  0.132431  1.140352  0.116132  0.907548   
24               opponent[T.Everton]  0.142934  0.980991  0.145703  0.884156   
25             opponent[T.Hull City]  1.134330  1.293705  0.876807  0.380591   
26        opponent[T.Leicester City]  0.102467  0.954432  0.107360  0.914504   
27             opponent[T.Liverpool]  0.976519  1.338159  0.729748  0.465544   
28       opponent[T.Manchester City] -0.457243  1.355663 -0.337284  0.735903   
29     opponent[T.Manchester United] -0.070679  1.364057 -0.051815  0.958676   
30      opponent[T.Newcastle United]  1.486382  1.315910  1.129547  0.258667   
31   opponent[T.Queens Park Rangers]  1.484756  1.331946  1.114726  0.264968   
32           opponent[T.Southampton] -0.235599  1.350654 -0.174433  0.861525   
33            opponent[T.Stoke City]  1.144134  1.377342  0.830683  0.406153   
34            opponent[T.Sunderland]  0.761103  1.274099  0.597365  0.550264   
35          opponent[T.Swansea City]  0.494079  1.282724  0.385180  0.700104   
36     opponent[T.Tottenham Hotspur]  0.607783  1.275396  0.476545  0.633686   
37  opponent[T.West Bromwich Albion]  0.820556  1.274368  0.643893  0.519645   
38       opponent[T.West Ham United]  1.014396  1.322619  0.766960  0.443105   
39                          f_HmGame  0.104651  0.303874  0.344390  0.730553   

      [0.025    0.975]  
0  -1.615630  2.422968  
1  -4.587240  0.337221  
2  -3.451611  1.093546  
3  -1.194462  1.493560  
4  -4.106254  0.803496  
5  -2.133871  1.385912  
6  -4.270269  0.812686  
7  -2.465749  1.173433  
8  -2.029277  2.015668  
9  -3.958862  0.991326  
10 -3.211516  1.024172  
11 -1.957473  1.636643  
12 -3.021539  1.034462  
13 -3.549545  1.266127  
14 -3.351986  1.238348  
15 -3.957041  0.990455  
16 -2.950005  1.361216  
17 -3.883296  1.070798  
18 -3.032323  1.312887  
19 -2.331287  1.407289  
20 -3.211755  2.399253  
21 -1.999327  2.319758  
22 -1.901639  1.844545  
23 -2.102618  2.367481  
24 -1.779774  2.065641  
25 -1.401285  3.669944  
26 -1.768185  1.973120  
27 -1.646224  3.599263  
28 -3.114294  2.199807  


Predict number of home goals

In [38]:
poisson_model.predict(pd.DataFrame(data={'f_Team': 'Arsenal', 'opponent': 'Southampton',
                                       'f_HmGame':1},index=[1]))

1    1.654825
dtype: float64

Output a matrix of score probabilities

In [39]:
def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'f_Team': homeTeam, 
                                                            'opponent': awayTeam,'f_HmGame':1},
                                                      index=[1])).values[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'f_Team': awayTeam, 
                                                            'opponent': homeTeam,'f_HmGame':0},
                                                      index=[1])).values[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [home_goals_avg, away_goals_avg]]
    return(np.outer(np.array(team_pred[0]), np.array(team_pred[1])))


In [41]:
from scipy.stats import poisson,skellam

In [70]:
simulate_match(poisson_model, 'Chelsea', 'Sunderland', max_goals=3)#[3][1]

array([[0.05857867, 0.02766916, 0.00653465, 0.00102886],
       [0.13854106, 0.06543875, 0.01545473, 0.00243331],
       [0.16382776, 0.07738272, 0.01827555, 0.00287744],
       [0.12915321, 0.06100448, 0.01440749, 0.00226842]])

Take the probabilities of the home team winning to get the probability for a Home Win

In [63]:
chel_sun = simulate_match(poisson_model, "Chelsea", "Sunderland", max_goals=10)
# chelsea win
np.sum(np.tril(chel_sun, -1))

0.7980995268261856

Probability of draw by taking the diagonal

In [64]:
np.sum(np.diag(chel_sun))

0.14472707782846353

probability of away win

In [67]:
np.sum(np.tril(chel_sun,-1))

0.7980995268261856

# Predicting all games

In [120]:
test_2014 = training[training["season"]==2014][["season","gw_no","matchdate","fixture","f_Team","f_HmGame","f_US xG",
                                               "f_Goals"]].copy()
test_2014.columns = test_2014.columns.str.replace(' ', '_')
def id_opponent(fixture,home):
    if home==1:
        opponent = fixture.split(" vs ")[1]
    else:
        opponent = fixture.split(" vs ")[0]
    return opponent
        
test_2014["opponent"] = test_2014.apply(lambda row: id_opponent(row["fixture"],row["f_HmGame"]),axis=1)
test_2014["Date"] = test_2014['matchdate'].apply(lambda x: x[0:10])
test_2014['Date'] = pd.to_datetime(test_2014['Date'])
test_2014['time_diff'] = (max(test_2014['Date']) - test_2014['Date']).dt.days
test_2014.head()

season  gw_no            matchdate                            fixture  \
0    2014      1  2014-08-16 12:45:00  Manchester United vs Swansea City   
1    2014      1  2014-08-16 12:45:00  Manchester United vs Swansea City   
2    2014      1  2014-08-16 15:00:00          Leicester City vs Everton   
3    2014      1  2014-08-16 15:00:00          Leicester City vs Everton   
4    2014      1  2014-08-16 15:00:00   Queens Park Rangers vs Hull City   

              f_Team  f_HmGame   f_US_xG  f_Goals             opponent  \
0       Swansea City         0  0.278076        2    Manchester United   
1  Manchester United         1  1.166350        1         Swansea City   
2            Everton         0  0.613273        2       Leicester City   
3     Leicester City         1  1.278300        2              Everton   
4          Hull City         0  1.117570        1  Queens Park Rangers   

        Date  time_diff  
0 2014-08-16        281  
1 2014-08-16        281  
2 2014-08-16        281  
3 2014-08-16        281  
4 2014-08-16        281

In [95]:
max_gw_date + pd.DateOffset(days=180)

Timestamp('2015-02-27 00:00:00')

In [121]:
for i in range(3,4):
    max_gw_date = test_2014[test_2014["gw_no"]==i]["Date"].max()
    print(max_gw_date)
    new_df = test_2014[(test_2014["Date"]<= max_gw_date) &
                      (test_2014["Date"]>=max_gw_date-pd.DateOffset(days=360) )]
    poisson_model = smf.glm(formula="f_US_xG ~ f_HmGame + f_Team + opponent", data=new_df, 
                        family=sm.families.Poisson()).fit()
    pred_df = test_2014[test_2014["gw_no"]==i+1].copy()
    pred_df["PredGoals"] = pred_df.apply(lambda row: 
              poisson_model.predict(pd.DataFrame(data={'f_Team': row["f_Team"],
                                                       'opponent': row["opponent"],
                                                       'f_HmGame':row["f_HmGame"]},index=[1])),axis=1)
pred_df.head()

2014-08-31 00:00:00


season  gw_no            matchdate                     fixture  \
60    2014      4  2014-09-13 12:45:00  Arsenal vs Manchester City   
61    2014      4  2014-09-13 12:45:00  Arsenal vs Manchester City   
62    2014      4  2014-09-13 15:00:00     Chelsea vs Swansea City   
63    2014      4  2014-09-13 15:00:00     Chelsea vs Swansea City   
64    2014      4  2014-09-13 15:00:00   Crystal Palace vs Burnley   

             f_Team  f_HmGame  f_US_xG  f_Goals         opponent       Date  \
60  Manchester City         0  2.30834        2          Arsenal 2014-09-13   
61          Arsenal         1  2.07804        2  Manchester City 2014-09-13   
62     Swansea City         0  1.02153        2          Chelsea 2014-09-13   
63          Chelsea         1  3.56581        4     Swansea City 2014-09-13   
64          Burnley         0  1.24930        0   Crystal Palace 2014-09-13   

    time_diff  PredGoals  
60        253   0.339562  
61        253   1.052404  
62        253   0.657525  
63        253   3.164353  
64        253   0.525749

In [143]:
season_list = [2014,2015,2016,2017]
season_gws = range(1,39)
final_order = pd.DataFrame()
x= 1
for i in season_list:
    for j in season_gws:
        gw_order=pd.DataFrame(data={"season":i,"gw_no":j,"gw_sq":x},index=[0])
        final_order = final_order.append(gw_order)
        x = x+1

In [159]:
train2 = pd.merge(training,final_order, on = ["season","gw_no"]).copy()
train2.tail()

season     gameweek           matchid  \
3035    2017  Gameweek 38  /matches/919270/   
3036    2017  Gameweek 38  /matches/919271/   
3037    2017  Gameweek 38  /matches/919271/   
3038    2017  Gameweek 38  /matches/919272/   
3039    2017  Gameweek 38  /matches/919272/   

                                  fixture            matchdate  \
3035           Swansea City vs Stoke City  2018-05-13 15:00:00   
3036  Tottenham Hotspur vs Leicester City  2018-05-13 15:00:00   
3037  Tottenham Hotspur vs Leicester City  2018-05-13 15:00:00   
3038           West Ham United vs Everton  2018-05-13 15:00:00   
3039           West Ham United vs Everton  2018-05-13 15:00:00   

                 f_Team  f_HmGame  f_Possession  f_Touches - Final Third  \
3035       Swansea City         1          56.8                      302   
3036     Leicester City         0          35.6                      116   
3037  Tottenham Hotspur         1          64.4                      169   
3038            Everton         0          43.2                      120   
3039    West Ham United         1          56.8                      204   

      f_Touches - Penalty Area  f_Passes - Final Third - Successful  \
3035                        45                                  139   
3036                        34                                   39   
3037                        25                                   72   
3038                        21                                   52   
3039                        27                                  111   

      f_Chances Created  f_Goals  f_Goals From Penalties  f_Goal Attempts  \
3035                 13        1                       0               27   
3036                 12        4                       0               17   
3037                 11        5                       0               14   
3038                  9        1                       0               14   
3039                 11        3                       0               15   

      f_Shots - Inside Box  f_Big Chances Total  f_Shots On Target  \
3035                    19                    4                 10   
3036                    14                    2                  9   
3037                     8                    2                  6   
3038                     9                    3                  7   
3039                     6                    1                  4   

      f_Goal Attempts In Box Conceded  f_Shots Conceded  \
3035                                5                 8   
3036                                8                14   
3037                               14                17   
3038                                6                15   
3039                                9                14   

      f_Shots On Target Conceded  f_Big Chances Conceded  f_xG Open Play  \
3035                           5                       3            2.58   
3036                           6                       2            0.81   
3037                           9                       2            1.55   
3038                           4                       1            1.27   
3039                           7                       3            0.99   

      f_xG Expected Goals  f_xA Expected Assists  f_Goals Conceded  \
3035                 3.00                   2.03                 2   
3036                 1.39                   1.15                 5   
3037                 1.76                   1.71                 4   
3038                 1.69                   0.59                 3   
3039                 1.13                   1.02                 1   

      f_xG Conceded  f_Touches - Final Third Conceded  \
3035           1.64                                60   
3036           1.76                               169   
3037           1.39                               116   
3038           1.13                               204   
3039           1.69                     

In [160]:
train2 = train2[["season","gw_no","matchdate","fixture","f_Team","f_HmGame","f_US xG", "f_Goals","gw_sq"]].copy()
train2.columns = train2.columns.str.replace(' ', '_')
def id_opponent(fixture,home):
    if home==1:
        opponent = fixture.split(" vs ")[1]
    else:
        opponent = fixture.split(" vs ")[0]
    return opponent
        
train2["opponent"] = train2.apply(lambda row: id_opponent(row["fixture"],row["f_HmGame"]),axis=1)
train2["Date"] = train2['matchdate'].apply(lambda x: x[0:10])
train2['Date'] = pd.to_datetime(train2['Date'])
train2['time_diff'] = (max(train2['Date']) - train2['Date']).dt.days
train2.head()

season  gw_no            matchdate                            fixture  \
0    2014      1  2014-08-16 12:45:00  Manchester United vs Swansea City   
1    2014      1  2014-08-16 12:45:00  Manchester United vs Swansea City   
2    2014      1  2014-08-16 15:00:00          Leicester City vs Everton   
3    2014      1  2014-08-16 15:00:00          Leicester City vs Everton   
4    2014      1  2014-08-16 15:00:00   Queens Park Rangers vs Hull City   

              f_Team  f_HmGame   f_US_xG  f_Goals  gw_sq             opponent  \
0       Swansea City         0  0.278076        2      1    Manchester United   
1  Manchester United         1  1.166350        1      1         Swansea City   
2            Everton         0  0.613273        2      1       Leicester City   
3     Leicester City         1  1.278300        2      1              Everton   
4          Hull City         0  1.117570        1      1  Queens Park Rangers   

        Date  time_diff  
0 2014-08-16       1366  
1 2014-08-16       1366  
2 2014-08-16       1366  
3 2014-08-16       1366  
4 2014-08-16       1366

In [167]:
train2[train2["f_Team"]=="Bournemouth"]

season  gw_no            matchdate                           fixture  \
763     2015      1  2015-08-08 18:00:00        Bournemouth vs Aston Villa   
798     2015      2  2015-08-17 23:00:00          Liverpool vs Bournemouth   
810     2015      3  2015-08-22 18:00:00    West Ham United vs Bournemouth   
825     2015      4  2015-08-29 18:00:00     Bournemouth vs Leicester City   
846     2015      5  2015-09-12 18:00:00       Norwich City vs Bournemouth   
...      ...    ...                  ...                               ...   
2938    2017     34  2018-04-14 16:30:00          Liverpool vs Bournemouth   
2951    2017     34  2018-04-18 18:45:00  Bournemouth vs Manchester United   
2978    2017     36  2018-04-28 14:00:00        Southampton vs Bournemouth   
2993    2017     37  2018-05-05 15:00:00       Bournemouth vs Swansea City   
3020    2017     38  2018-05-13 15:00:00            Burnley vs Bournemouth   

           f_Team  f_HmGame   f_US_xG  f_Goals  gw_sq           opponent  \
763   Bournemouth         1  0.876106        0     39        Aston Villa   
798   Bournemouth         0  0.460805        0     40          Liverpool   
810   Bournemouth         0  2.855400        4     41    West Ham United   
825   Bournemouth         1  0.321432        1     42     Leicester City   
846   Bournemouth         0  0.389495        1     43       Norwich City   
...           ...       ...       ...      ...    ...                ...   
2938  Bournemouth         0  0.540593        0    148          Liverpool   
2951  Bournemouth         1  1.183010        0    148  Manchester United   
2978  Bournemouth         0  1.017410        1    150        Southampton   
2993  Bournemouth         1  1.696180        1    151       Swansea City   
3020  Bournemouth         0  1.385510        2    152            Burnley   

           Date  time_diff  
763  2015-08-08       1009  
798  2015-08-17       1000  
810  2015-08-22        995  
825  2015-08-29        988  
846  2015-09-12        974  
...         ...        ...  
2938 2018-04-14         29  
2951 2018-04-18         25  
2978 2018-04-28         15  
2993 2018-05-05          8  
3020 2018-05-13          0  

[114 rows x 12 columns]

In [178]:
final_df = pd.DataFrame()
for i in range(10,152):
    max_gw_date = train2[train2["gw_sq"]==i]["Date"].max()
    predict_gw = train2[train2["gw_sq"]==i+1]["gw_no"].max()
    new_df = train2[(train2["Date"]<= max_gw_date) &
                      (train2["Date"]>=max_gw_date-pd.DateOffset(days=360) )]
    if predict_gw <=3:
        continue
        
    poisson_model = smf.glm(formula="f_Goals ~ f_HmGame + f_Team + opponent", data=new_df, 
                        family=sm.families.Poisson()).fit()
    pred_df = train2[train2["gw_sq"]==i+1].copy()
    pred_df["PredGoals"] = pred_df.apply(lambda row: 
              poisson_model.predict(pd.DataFrame(data={'f_Team': row["f_Team"],
                                                       'opponent': row["opponent"],
                                                       'f_HmGame':row["f_HmGame"]},index=[1])),axis=1)
    final_df = pred_df.append(pred_df)
final_df.tail()

season  gw_no            matchdate                              fixture  \
3035    2017     38  2018-05-13 15:00:00           Swansea City vs Stoke City   
3036    2017     38  2018-05-13 15:00:00  Tottenham Hotspur vs Leicester City   
3037    2017     38  2018-05-13 15:00:00  Tottenham Hotspur vs Leicester City   
3038    2017     38  2018-05-13 15:00:00           West Ham United vs Everton   
3039    2017     38  2018-05-13 15:00:00           West Ham United vs Everton   

                 f_Team  f_HmGame  f_US_xG  f_Goals  gw_sq           opponent  \
3035       Swansea City         1  1.70936        1    152         Stoke City   
3036     Leicester City         0  1.27359        4    152  Tottenham Hotspur   
3037  Tottenham Hotspur         1  1.83248        5    152     Leicester City   
3038            Everton         0  1.67867        1    152    West Ham United   
3039    West Ham United         1  1.18299        3    152            Everton   

           Date  time_diff  PredGoals  
3035 2018-05-13          0   1.122797  
3036 2018-05-13          0   0.794618  
3037 2018-05-13          0   2.669493  
3038 2018-05-13          0   1.362455  
3039 2018-05-13          0   1.613856

In [179]:
metrics.mean_squared_error(final_df["f_Goals"],final_df["PredGoals"])

1.948757399639873

In [ ]:
fin